In [1]:
import tensorflow as tf

model = tf.keras.models.load_model('model.h5', compile=False)

In [2]:
import numpy as np
def detect(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (256, 256))
    image = image.astype(np.float32) / 255.0 
    image = np.expand_dims(image, axis=-1)
    image = np.expand_dims(image, axis=0)
    predictions = model.predict(image)
    predicted_class = np.argmax(predictions, axis=1)
    l = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
    return(l[predicted_class[0]+1])

In [7]:
def draw_label(img, text, pos, bg_color):
    text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1, cv2.FILLED)
    
    end_x = pos[0] + text_size[0][0] + 2
    end_y = pos[1] + text_size[0][1] - 2
    
    
    cv2.putText(img, text, pos, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), thickness=1, lineType=cv2.LINE_AA)

In [12]:
import cv2
import mediapipe as mp
import time

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

cap = cv2.VideoCapture(0)
hands = mp_hands.Hands()
start_time = time.time()

while True:
    success, image = cap.read()

    image = cv2.flip(image, 1)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    results = hands.process(image)

    x_min, y_min = float('inf'), float('inf')
    x_max, y_max = float('-inf'), float('-inf')

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
          
            for lm in hand_landmarks.landmark:
                x, y = int(lm.x * image.shape[1]), int(lm.y * image.shape[0])
                x_min = min(x_min, x)
                y_min = min(y_min, y)
                x_max = max(x_max, x)
                y_max = max(y_max, y)

            cv2.rectangle(image, (x_min + 10, y_min + 10), (x_max + 10, y_max + 10), (0, 255, 0), 2)
            roi = image[y_min:y_max, x_min:x_max]
            
        
            
            y_pred = detect(roi)
        
    
            draw_label(image, y_pred, (50,50), (255,0,0))

#             mp_drawing.draw_landmarks(
#                 image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
    
    
    
    cv2.imshow('MediaPipe Hands', image)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    current_time = time.time()
    if current_time - start_time >= 5:
        cv2.imwrite('screenshot.png', image)
        print("Screenshot taken!")
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 25ms/step
Screenshot taken!
